<a href="https://colab.research.google.com/github/Abhinayy27/NLP-Project/blob/main/main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

path ='/content/drive/MyDrive/ML/Blackcopper Test/Input.xlsx'
input_df = pd.read_excel(path)

# Function to clean extracted text
def clean_extracted_text(text):
    patterns_to_remove = [
        r"Firm Name:.*",                # Remove firm details
        r"Email:.*",                    # Remove email addresses
        r"Skype:.*",                    # Remove Skype details
        r"WhatsApp:.*",                 # Remove WhatsApp details
        r"We provide.*",                # Remove boilerplate about services
        r"©.*All Right Reserved.*",     # Remove legal copyright information
        r"Summarized:.*",               # Remove summarization text
        r"AI and ML-Based YouTube Analytics.*"  # Remove irrelevant repeated content
    ]

    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text, flags=re.DOTALL)

    # Further cleaning: remove multiple newlines or excessive spaces
    text = re.sub(r'\n+', '\n', text).strip()

    return text

# Function to extract content from the <div> with class "td-post-content tagdiv-type"
def extract_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Target the specific div container by its class
        main_content = soup.find('div', {'class': 'td-post-content tagdiv-type'})

        if main_content:
            # Extract title from <h1> tag
            title = soup.find('h1').get_text().strip() if soup.find('h1') else 'No title'

            # Extract paragraphs, ordered lists, list items, and other heading tags inside the main content
            content_parts = []

            # Extract paragraphs (<p>), ordered lists (<ol> and <li>), and headers (<h2>, <h3>, etc.)
            for tag in main_content.find_all(['p', 'ol', 'li', 'h1', 'h2', 'h3', 'h4']):
                content_parts.append(tag.get_text().strip())

            # Join all parts together
            article_text = '\n'.join(content_parts)

            cleaned_text = clean_extracted_text(article_text)
            return title, cleaned_text
        else:
            print(f"Main content not found for {url}")
            return None, None
    except Exception as e:
        print(f"Failed to extract from {url}: {e}")
        return None, None

# Output folder to store the files
output_folder = '/content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts'
os.makedirs(output_folder, exist_ok=True)

# Iterate through all URLs and save extracted text into a file
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    title, text = extract_text(url)

    if title and text:
      try:
        # Save the text to a file with URL_ID as the file name in the output folder
        file_path = os.path.join(output_folder, f'{url_id}.txt')
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(f"Title: {title}\n\n{text}")
        print(f"Successfully saved: {file_path}")
      except Exception as e:
            print(f"Error writing file {url_id}: {e}")
    else:
        print(f"Failed to extract content for URL_ID {url_id}")


Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2011.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2012.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2013.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2014.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2015.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2016.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2017.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2018.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2019.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts/bctech2020.txt
Successfully saved: /content/drive/MyDrive/ML/Blackcopper Te

In [ ]:
pip install beautifulsoup4 nltk textstat


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import re

# Download NLTK tokenizer if not already downloaded
nltk.download('punkt')

# Function to load stopwords from multiple files
def load_stop_words(stop_words_folder):
    stop_words = set()
    for filename in os.listdir(stop_words_folder):
        if filename.endswith('.txt'):
            file_path = os.path.join(stop_words_folder, filename)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                    words = file.read().splitlines()
                    stop_words.update(words)
            except UnicodeDecodeError as e:
                print(f"Error decoding {filename}: {e}")
    return stop_words

# Load positive and negative word dictionaries
def load_word_list(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
            return set(file.read().splitlines())
    except UnicodeDecodeError as e:
        print(f"Error decoding {filepath}: {e}")
        return set()

# Load the stop words, positive, and negative word lists
stop_words_folder = '/content/drive/MyDrive/ML/Blackcopper Test/StopWords'
positive_words_path = '/content/drive/MyDrive/ML/Blackcopper Test/MasterDictionary/positive-words.txt'
negative_words_path = '/content/drive/MyDrive/ML/Blackcopper Test/MasterDictionary/negative-words.txt'

stop_words = load_stop_words(stop_words_folder)
positive_words = load_word_list(positive_words_path)
negative_words = load_word_list(negative_words_path)

# Function to count syllables in a word
def count_syllables(word):
    word = word.lower()
    vowels = "aeiou"
    syllables = 0
    for i, letter in enumerate(word):
        if letter in vowels:
            if i == 0 or word[i - 1] not in vowels:
                syllables += 1
    if word.endswith(("es", "ed")) and syllables > 1:
        syllables -= 1
    return syllables

# Function to count personal pronouns
def count_personal_pronouns(text):
    pronoun_pattern = r'\b(I|we|my|ours|us)\b'
    pronouns = re.findall(pronoun_pattern, text, re.I)
    return len(pronouns)

# Text analysis function
def text_analysis(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]

    positive_score = sum(1 for word in filtered_words if word in positive_words)
    negative_score = sum(1 for word in filtered_words if word in negative_words) * -1
    polarity_score = (positive_score - abs(negative_score)) / ((positive_score + abs(negative_score)) + 0.000001)
    subjectivity_score = (positive_score + abs(negative_score)) / (len(filtered_words) + 0.000001)

    word_count = len(filtered_words)
    syllable_count = sum([count_syllables(word) for word in filtered_words])
    syllables_per_word = syllable_count / word_count if word_count != 0 else 0
    avg_word_length = sum(len(word) for word in filtered_words) / word_count if word_count != 0 else 0

    # Analysis of Readability (Gunning Fog Index)
    avg_sentence_length = word_count / len(sentences) if len(sentences) > 0 else 0
    complex_word_count = sum(1 for word in filtered_words if count_syllables(word) > 2)
    percentage_complex_words = (complex_word_count / word_count) * 100 if word_count != 0 else 0
    fog_index = 0.4 * (avg_sentence_length + (complex_word_count / word_count) if word_count != 0 else 0)

    # Personal pronouns count
    personal_pronouns_count = count_personal_pronouns(text)

    # Calculate average words per sentence
    avg_words_per_sentence = word_count / len(sentences) if len(sentences) > 0 else 0

    return {
        'Positive Score': positive_score,
        'Negative Score': abs(negative_score),
        'Polarity Score': polarity_score,
        'Subjectivity Score': subjectivity_score,
        'Word Count': word_count,
        'Syllables per Word': syllables_per_word,
        'Avg Word Length': avg_word_length,
        'Complex Word Count': complex_word_count,
        'Percentage of Complex Words': percentage_complex_words,
        'Fog Index': fog_index,
        'Avg Sentence Length': avg_sentence_length,
        'Avg Words Per Sentence': avg_words_per_sentence,
        'Personal Pronouns': personal_pronouns_count
    }

# Main function to update the existing Excel with analysis results
def update_existing_excel(existing_excel_path, folder_path, output_file):
    # Load existing Excel file
    df = pd.read_excel(existing_excel_path)

    # Iterate through the rows of the Excel to perform text analysis for each ID
    analysis_data = []
    for index, row in df.iterrows():
        file_id = row['URL_ID']  # Assuming 'ID' column in the Excel file matches .txt file names
        file_path = os.path.join(folder_path, f'{file_id}.txt')

        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

                # Perform text analysis
                analysis_results = text_analysis(text)

                # Append results to a list
                analysis_data.append({**row.to_dict(), **analysis_results})
        else:
            print(f"File for ID {file_id} not found.")

    # Create a new DataFrame with the updated results
    updated_df = pd.DataFrame(analysis_data)

    # Save the updated DataFrame back to a new Excel file
    updated_df.to_excel(output_file, index=False)

# Paths
existing_excel_path = '/content/drive/MyDrive/ML/Blackcopper Test/Output Data Structure.xlsx'  # Existing Excel file with ID-URL mappings
folder_path = '/content/drive/MyDrive/ML/Blackcopper Test/Extracted_Texts'  # Folder with .txt files
output_file = '/content/drive/MyDrive/ML/Blackcopper Test/Updated_Text_Analysis_Results.xlsx'

# Run the analysis and update the Excel file
update_existing_excel(existing_excel_path, folder_path, output_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
